# Potential and current in a (classical) Hall bar

FEniCS is the open-source finite element computing and visuable platform for solving partial differential equations (PDEs) using a high-level, variational(weak-form) formulation.

Installation instructions for fenics is 
https://fenicsproject.org/download/

and the legacy is https://fenicsproject.org/download/archive/

`conda create -n fenicsx python=3.10 -c conda-forge`

`conda activate fenicsx`

`conda install fenics-dolfinx`

`python -m ipykernel install --user --name fenicsx --display-name "Python (fenicsx)"`

we use the fenics(legacy) here.


In [1]:
#! /usr/bin/env python3
# based on demo_mixed-poisson.py
# but see sections 4.1 and 4.2 in the tutorial pdf
# and https://fenicsproject.discourse.group/t/pde-system-simple-example/1550/2

# now also section 4.3 and https://fenicsproject.org/docs/dolfin/2019.1.0/python/demos/tensor-weighted-poisson/documentation.html

# Installation instructions for fenics: https://fenicsproject.org/download/archive/
# This script works with version 2019.1.0

from dolfin import *
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
BarLength = 870.0
BarWidth = 220.0

ILead1 = 0.0
ILead2 = 220.0

VLead1 = 280.0
VLead2 = 310.0

VLead3 = 560.0
VLead4 = 590.0

ResFactor = 2

LPoints = ResFactor*87
WPoints = ResFactor*22

ResL = BarLength/LPoints
ResW = BarWidth/WPoints

mu = 6 # m2/Vs
ns = 7e15 # m-2
qe = 1.602e-19 # C

B = 1.0 # T
# B = 1/mu # equipotentials are at 45 degrees

cond0 = ns*qe*mu
rho0 = 1/cond0
#print('Zero-field conductivity: {:.2e}'.format(cond0))
#print('Zero-field resistivity: {:.2f} Ohm/sq'.format(rho0))

R2 = rho0*BarLength/BarWidth
RH = 1/(qe*ns)
print('Two-terminal zero-field resistance {:.2f} Ohm'.format(R2))
print('... at B = {:.2f} T:'.format(B))
print('Expected Hall coefficient: {:.2f} Ohm/T'.format(RH))
print('rho_xx = {:.2f} Ohm/sq\trho_xy = {:.2f} Ohm'.format(rho0,B*RH))

# print('{:.2f}\t{:.2f}\t{:.2f}\t{:.2f}\t{:.2f}\t{:.4e}\t{:.4f}'.format(B,R2,rho0,B*RH,RH,ns,mu))


# Create mesh
mesh = RectangleMesh(Point(0.0, 0.0), Point(BarLength, BarWidth), LPoints, WPoints, "right/left")


el_scalar = FiniteElement("Lagrange", mesh.ufl_cell(), 1)  # potential
el_vector = VectorElement("Lagrange", mesh.ufl_cell(), 2)  # current density
el = MixedElement([el_scalar, el_vector])
Vmixed = FunctionSpace(mesh, el)
v, tau = TestFunctions(Vmixed)
u, sigma = TrialFunctions(Vmixed)

phi = Vmixed.sub(0)
J = Vmixed.sub(1)

# Semiconductor
csemixx = cond0/(1+mu*mu*B*B)
csemixy = cond0*mu*B/(1+mu*mu*B*B)
# Voltage probes
ccontact = Constant(1000.0)
ccontactH = Constant(0.0)
f = Constant(0.0)

class Sxx(UserExpression):
    def set_cond_values(self, csemixx, ccontact):
        self.csemixx, self.ccontact = csemixx, ccontact

    def eval(self, value, x):
        if x[0] > VLead1 and x[0] < VLead2 and (x[1] < 0.0 + ResW or x[1] > BarWidth - ResW):
            value[0] = self.ccontact

        elif x[0] > VLead3 and x[0] < VLead4 and (x[1] < 0.0 + ResW or x[1] > BarWidth - ResW):
            value[0] = self.ccontact

        else:
            value[0] = self.csemixx

class Sxy(UserExpression):
    def set_cond_values(self, csemixy, ccontactH):
        self.csemixy, self.ccontactH = csemixy, ccontactH

    def eval(self, value, x):
        if x[0] > VLead1 and x[0] < VLead2 and (x[1] < 0.0 + ResW or x[1] > BarWidth - ResW):
            value[0] = self.ccontactH

        elif x[0] > VLead3 and x[0] < VLead4 and (x[1] < 0.0 + ResW or x[1] > BarWidth - ResW):
            value[0] = self.ccontactH

        elif x[1] > ILead1 and x[1] < ILead2 and (x[0] < 0.0 + ResL or x[0] > BarLength - ResL):
            value[0] = self.ccontactH

        else:
            value[0] = self.csemixy

#Initialize conductivity
condxx = Sxx(mesh, degree=0)
condxy = Sxy(mesh, degree=0)
condxx.set_cond_values(csemixx,ccontact)
condxy.set_cond_values(csemixy,ccontactH)

cond = as_matrix(((condxx,condxy),(-condxy,condxx)))

#cond = as_matrix(((S[0],S[1]),(-S[1],S[0])))

F = dot(cond*grad(u) + sigma, tau) * dx + dot(sigma, grad(v)) * dx - f * v * dx

# Define function G such that G \cdot n = g
class BoundarySource(UserExpression):
    def __init__(self, mesh, **kwargs):
        self.mesh = mesh
        super().__init__(**kwargs)
    def eval_cell(self, values, x, ufc_cell):
        cell = Cell(self.mesh, ufc_cell.index)
        n = cell.normal(ufc_cell.local_facet)
        g = 0.0
        values[0] = g*n[0]
        values[1] = g*n[1]
    def value_shape(self):
        return (2,)

G = BoundarySource(mesh, degree=2)

# Define essential boundary
def boundary(x, on_boundary):
    return on_boundary # (x[1] < 0.0 + ResW or x[1] > BarWidth - ResW)

bc_Sides = DirichletBC(J, G, boundary, method='pointwise')

# Define left and right potentials

VD = 0.010

u_L = Constant(VD)
u_R = Constant(0.000)

def boundary_L(x, on_boundary):
    return (x[0] < 0.0 + ResL) and (x[1] > ILead1 and x[1] < ILead2)

def boundary_R(x, on_boundary):
    return (x[0] > BarLength - ResL) and (x[1] > ILead1 and x[1] < ILead2)

bc_L = DirichletBC(phi, u_L, boundary_L, method='pointwise')
bc_R = DirichletBC(phi, u_R, boundary_R, method='pointwise')

# Collect boundary conditions

bc = [bc_L, bc_R, bc_Sides]

u = Function(Vmixed)

solve(lhs(F) == rhs(F), u, bc)

phi, J = u.split()

# calculate total current flux in x direction

total_current = assemble(J[0]*dx)
print('Potential difference = {} V, total current = {} A, actual 2-terminal resistance = {} Ohm'.format(VD,total_current,VD/total_current))
# print(total_current)

plt.figure('current density')
plot(J)

xhat = as_vector((1.0,0.0))
yhat = as_vector((0.0,1.0))
Jx = dot(J, xhat)
Jy = dot(J, yhat)
Jx_proj = project(Jx, FunctionSpace(mesh, el_scalar))
Jy_proj = project(Jy, FunctionSpace(mesh, el_scalar))

# Save potential
phi_filename='fenics-Hall-bar_phi'
phi_file = File(phi_filename+'.xyz')
phi_file << phi

# Save J
#J_file = File("bar08-2_J.pvd")
#J_file << J

# Save Jx and Jy
Jx_filename='fenics-Hall-bar_Jx'
Jy_filename='fenics-Hall-bar_Jy'
Jx_file = File(Jx_filename+'.xyz')
Jy_file = File(Jy_filename+'.xyz')
Jx_file << Jx_proj
Jy_file << Jy_proj

plt.figure('current density in x direction')
plot(Jx)
plt.figure('current density in y direction')
plot(Jy)

plt.figure('potential')
plot(phi)

plt.show()

In [ ]:
phi_filedata=phi_filename+'000000.xyz'
Jx_filedata=Jx_filename+'000000.xyz'
Jy_filedata=Jy_filename+'000000.xyz'

phix, phiy, phival = np.loadtxt(phi_filedata, unpack=True)
Jxx, Jxy, Jxval = np.loadtxt(Jx_filedata, unpack=True)

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

def getPotential(position):
    #print(position)
    possiblex = np.where(np.abs(phix - position[0])<0.5*ResL)
    #print(possiblex[0])
    possibley = np.where(np.abs(phiy - position[1])<0.5*ResW)
    #print(possibley[0])
    onlyxy = np.intersect1d(possiblex,possibley)
    return phival[onlyxy]

def getCurrent(position):
    #print(position)
    line = np.where(np.abs(Jxx - position)<0.5*ResL)
    Jxline = Jxval[line]
    Jxposy = Jxy[line]
    #print(Jxposy)
    Itot = np.trapz(Jxline,x=Jxposy)
    return Itot

probex12 = 0.5*(VLead1+VLead2)
probex34 = 0.5*(VLead3+VLead4)

# voltage probe positions:
probe1 = np.array((probex12,0.0))
probe2 = np.array((probex12,BarWidth))
probe3 = np.array((probex34,0.0))
probe4 = np.array((probex34,BarWidth))

V1 = getPotential(probe1)[0]
V2 = getPotential(probe2)[0]
V3 = getPotential(probe3)[0]
V4 = getPotential(probe4)[0]

Vx1 = V1-V3
Vx2 = V2-V4
Vy1 = V1-V2
Vy2 = V3-V4

# current integration line position in x:
currentx = 0.5*BarLength

Idrain = getCurrent(currentx)
print('current = {:.2f} µA'.format(1e6*Idrain))

R2 = VD/Idrain

print('length/width = {:.4f}'.format((probex34-probex12)/BarWidth))

rhoxx1 = Vx1/Idrain * (BarWidth / (probex34-probex12))
rhoxx2 = Vx2/Idrain * (BarWidth / (probex34-probex12))
rhoxx = 0.5*(rhoxx1+rhoxx2)
rhoxy1 = Vy1/Idrain
rhoxy2 = Vy2/Idrain
rhoxy = 0.5*(rhoxy1+rhoxy2)
#print(rhoxx1)
#print(rhoxx2)
#print(rhoxy1)
#print(rhoxy2)
print('Two-terminal resistance {:.2f} Ohm'.format(R2))
print('rho_xx = {:.2f} Ohm/sq\trho_xy = {:.2f} Ohm'.format(rhoxx,rhoxy))

if B != 0.0:
    RH = rhoxy/B
    ns = 1/(qe*RH)
    print('... at B = {:.2f} T:'.format(B))
    print('Hall coefficient: {:.2f} Ohm/T'.format(RH))
    print('Hall sheet density: {:.4e} cm-2'.format(ns*1e-4))
    mu = RH/rhoxx
    print('Hall mobility: {:.0f} cm2/Vs'.format(1e4*mu))
    # print('{:.2f}\t{:.2f}\t{:.2f}\t{:.2f}\t{:.2f}\t{:.4e}\t{:.4f}'.format(B,R2,rhoxx,rhoxy,RH,ns,mu))

else:
    print()
    #print('{:.2f}\t{:.2f}\t{:.2f}\t{:.2f}'.format(B,R2,rhoxx,rhoxy))
current = 7.25 µA
length/width = 1.2727
Two-terminal resistance 1379.90 Ohm
rho_xx = 149.43 Ohm/sq	rho_xy = 860.84 Ohm
... at B = 1.00 T:
Hall coefficient: 860.84 Ohm/T
Hall sheet density: 7.2513e+11 cm-2
Hall mobility: 57609 cm2/Vs
